## Introduction

## Importing Necessary Libraries

In [177]:
!pip install matplotlib

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import Image

import ipywidgets as widgets
from IPython.display import display, clear_output

from math import * 
from dataclasses import dataclass, asdict
from enum import Enum
from typing import List 
import random
import pandas as pd
import time


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


## Defining Constants and Variables

In [114]:
class Basis(Enum):
    Rectilinear = ("H", "V")
    Diagonal = ("D", "A")

class States(Enum):
    H = (1, 0)
    V = (0, 1)
    D = (1/1/sqrt(2), 1/1/sqrt(2))
    A = (1/1/sqrt(2), -1/1/sqrt(2))
    
    @property
    def basis(self):
        return Basis.Rectilinear if self.name in Basis.Rectilinear.value else Basis.Diagonal
    
    @property
    def bit(self):
        return 0 if self.name in ("H", "D") else 1

@dataclass
class Measurement:
    d1h: bool
    d1v: bool
    d2h: bool
    d2v: bool
    
    @property
    def bell_plus(self):
        return (self.d1h & self.d1v) | (self.d2h & self.d2v)
    
    @property
    def bell_minus(self):
        return (self.d1h & self.d2v) | (self.d1v & self.d2h)
    
@dataclass
class SimulationResult:
    alice_state: str
    alice_bit: int
    bob_state: str
    bob_bit: int
    
    basis: str
    bell_plus: bool
    bell_minus: bool
    is_valid: bool

## Defining Helper Functions

#### Expected Photon Count

In [115]:
N1v = lambda ah, av, bh, bv: (av - bv)**2 / 2
N1h = lambda ah, av, bh, bv: (ah - bh)**2 / 2
N2v = lambda ah, av, bh, bv: (av + bv)**2 / 2
N2h = lambda ah, av, bh, bv: (ah + bh)**2 / 2

#### Simulation

In [197]:
def simulate_measurement(alice: States, bob: States) -> Measurement:
    
    detector_weights = [
        N1v(*alice.value, *bob.value),
        N1h(*alice.value, *bob.value),
        N2v(*alice.value, *bob.value),
        N2h(*alice.value, *bob.value),
    ]
    
    detector_hits = random.choices(["d1v", "d1h", "d2v", "d2h"], weights=detector_weights, k=2)
    
    d1v = "d1v" in detector_hits
    d1h = "d1h" in detector_hits
    d2v = "d2v" in detector_hits
    d2h = "d2h" in detector_hits
    
    return Measurement(d1h=d1h, d1v=d1v, d2h=d2h, d2v=d2v)


def simulate_experiment(steps: int) -> List[SimulationResult]:
    
    results = []
    
    for step in range(steps):
        alice = random.choice(list(States))
        bob = random.choice(list(States))
        measurement = simulate_measurement(alice, bob)
        is_same_basis = alice.basis == bob.basis
        basis = alice.basis.name if is_same_basis else "n/a"
        
        is_valid = (measurement.bell_plus | measurement.bell_minus) & is_same_basis
        
        
        simulation_result = SimulationResult(
            alice_state=alice.name, 
            alice_bit=alice.bit, 
            bob_state=bob.name, 
            bob_bit=bob.bit, 
            basis=basis, 
            bell_plus=measurement.bell_plus, 
            bell_minus=measurement.bell_minus,
            is_valid=is_valid
        )
        results.append(simulation_result)
        
    return results


def apply_bit_flip(results_df: pd.DataFrame) -> pd.DataFrame:
    
    if results_df["basis"] == Basis.Rectilinear.name:
        return (results_df["bob_bit"] + 1) % 2
    
    if results_df["basis"] == Basis.Diagonal.name and results_df["bell_minus"]:
        return (results_df["bob_bit"] + 1) % 2

    return results_df["bob_bit"]


def generate_key(target_key_size: int):
    
    key_rate = 0.19
    multiplier = 1 / key_rate
    
    steps = int(target_key_size * multiplier)
    results = simulate_experiment(steps)
    df = pd.DataFrame(results)
    df = df[df["is_valid"]]
    df["bob_bit_sifted"] = df.apply(apply_bit_flip, axis=1)
    
    alice_key = ''.join(df['alice_bit'].astype(str))
    bob_key = ''.join(df['bob_bit_sifted'].astype(str))
    
    alice_key_hex = hex(int(alice_key, 2))
    bob_key_hex = hex(int(bob_key, 2))

    assert alice_key_hex == bob_key_hex, "Keys do not match"
    return alice_key, alice_key_hex

## Simulating Experiment

In [202]:
desired_key_size = 100
key_binary, key_hex = generate_key(desired_key_size)
key_size = len(key_binary)

print(f"{key_binary=}")
print(f"{key_hex=}")
print(f"{key_size=}")

key_binary='1000101100001111111000110101100000011111001010101110010011000000010100100010011010110000000110101011111101'
key_hex='0x22c3f8d607cab9301489ac06afd'
key_size=106


### Testing

In [157]:
for alice in States:
    for bob in States:
        
        same_basis = alice.basis == bob.basis
        
        print(f"{alice.name}, {bob.name}: {same_basis}")

H, H: True
H, V: True
H, D: False
H, A: False
V, H: True
V, V: True
V, D: False
V, A: False
D, H: False
D, V: False
D, D: True
D, A: True
A, H: False
A, V: False
A, D: True
A, A: True


In [158]:
steps = 100000


outputs = {
    "Alice": [],
    "Bob": [],
    "d1v": [],
    "d1h": [],
    "d2v": [],
    "d2h": []
}

for alice in States:
    for bob in States:
        
        d1 = 0
        d2 = 0
        d3 = 0
        d4 = 0
        
        for i in range(steps):
            detector_weights = [
                N1v(*alice.value, *bob.value),
                N1h(*alice.value, *bob.value),
                N2v(*alice.value, *bob.value),
                N2h(*alice.value, *bob.value),
            ]
    
            detector_hits = random.choices(["d1v", "d1h", "d2v", "d2h"], weights=detector_weights, k=2)
            d1 += detector_hits.count("d1v")
            d2 += detector_hits.count("d1h")
            d3 += detector_hits.count("d2v")
            d4 += detector_hits.count("d2h")

        
        outputs["Alice"].append(alice)
        outputs["Bob"].append(bob)


        outputs['d1v'].append(round(d1/steps, 2))
        outputs['d1h'].append(round(d2/steps, 2))
        outputs['d2v'].append(round(d3/steps, 2))
        outputs['d2h'].append(round(d4/steps, 2))
        
df = pd.DataFrame(outputs)  
df = df.set_index(["Alice", "Bob"])  

df

d1v   d1h   d2v   d2h
Alice    Bob                             
States.H States.H  0.00  0.00  0.00  2.00
         States.V  0.50  0.50  0.50  0.50
         States.D  0.25  0.04  0.25  1.46
         States.A  0.25  0.04  0.25  1.46
States.V States.H  0.50  0.50  0.50  0.50
         States.V  0.00  0.00  2.00  0.00
         States.D  0.04  0.25  1.46  0.25
         States.A  1.46  0.25  0.04  0.25
States.D States.H  0.25  0.04  0.25  1.45
         States.V  0.04  0.25  1.46  0.25
         States.D  0.00  0.00  1.00  1.00
         States.A  1.00  0.00  0.00  1.00
States.A States.H  0.25  0.04  0.25  1.46
         States.V  1.46  0.25  0.04  0.25
         States.D  1.00  0.00  0.00  1.00
         States.A  0.00  0.00  1.00  1.00

In [181]:
key_rates = []

start = time.time()
for step in range(100, 10000, 100):
    
    results = simulate_experiment(step)

    df = pd.DataFrame(results)

    df = df[df["is_valid"]]
    
    # print(step, df.shape[0], round(df.shape[0] / step, 2))
    
    key_rates.append(df.shape[0] / step)

end = time.time()

avg_key_rate = round(np.mean(key_rates), 2) 
std_key_rate = round(np.std(key_rates), 2)

print(f"{avg_key_rate=}")
print(f"{std_key_rate=}")
print(f"Time to run: {round(end-start, 2)} seconds")

avg_key_rate=0.19
std_key_rate=0.01
Time to run: 29.22 seconds
